In [1]:
import os, shutil, json
import pandas as pd
import numpy as np
import scipy.io.wavfile as sc_wav

In [2]:
MAIN_PATH = "gridspace-stanford-harper-valley/data"
SOUNDS_PATH = os.path.join(MAIN_PATH, "audio")

df = pd.read_csv("selected_samples.csv")
df.head(2)

,Unnamed: 0,transcript_path,agent_positive_min,agent_positive_max,agent_positive_mean,agent_neutral_min,agent_neutral_max,agent_neutral_mean,agent_negative_min,agent_negative_max,...,caller_negative_min,caller_negative_max,caller_negative_mean,caller_positive_end,metadata_path,task_type,end_time_ms,sid,start_time_ms,code
0,8c8a5b0505d74c5d.json,gridspace-stanford-harper-valley/data/transcri...,0.0,0.719676,0.252588,0.268360,1.0,0.663952,0.0,0.332475,...,0.206118,0.603673,0.328819,0,gridspace-stanford-harper-valley/data/metadata...,order checks,1591055980580,8c8a5b0505d74c5d,1591055932101,0
1,d902244660d44131.json,gridspace-stanford-harper-valley/data/transcri...,0.0,0.706142,0.292093,0.284698,1.0,0.537955,0.0,0.529775,...,0.054305,0.278522,0.164718,0,gridspace-stanford-harper-valley/data/metadata...,order checks,1591060421582,d902244660d44131,1591060353168,10


In [ ]:
#shutil.rmtree('dataset')

In [4]:
## 1. Copy all of the selected ones to the main path:
NEW_PATH = "dataset"
NEW_TRANSCRIPT_PATH = os.path.join(NEW_PATH, "transcript")
NEW_METADATA_PATH = os.path.join(NEW_PATH, "metadata")

NEW_SOUNDS_PATH = os.path.join(NEW_PATH, "audio")
AGENT_PATH = os.path.join(NEW_SOUNDS_PATH, 'agent')
CALLER_PATH = os.path.join(NEW_SOUNDS_PATH, 'caller')
MIXED_PATH = os.path.join(NEW_PATH, 'mixed')

for pth in [NEW_PATH, NEW_TRANSCRIPT_PATH, NEW_METADATA_PATH, NEW_SOUNDS_PATH, 
            AGENT_PATH, CALLER_PATH, MIXED_PATH]:
    if not os.path.exists(pth): os.mkdir(pth)

for idx, row in df.iterrows():
    sid = row['sid']
    # copy metadata:
    shutil.copy(row['metadata_path'], os.path.join(NEW_METADATA_PATH, f"{sid}.json"))
    #copy transcript:
    shutil.copy(row['transcript_path'], os.path.join(NEW_TRANSCRIPT_PATH, f"{sid}.json"))
    #copy audios 
    shutil.copy(os.path.join(SOUNDS_PATH, 'agent', f"{sid}.wav"), os.path.join(AGENT_PATH, f"{sid}.wav"))
    shutil.copy(os.path.join(SOUNDS_PATH, 'caller', f"{sid}.wav"), os.path.join(CALLER_PATH, f"{sid}.wav"))

In [5]:
met_path = os.path.join(MAIN_PATH, 'metadata')
cc = 0
kk = 0
null_hg, caller_hg_first = 0, 0
for fn in os.listdir(met_path):
    meta_json = os.path.join(met_path, fn)
    with open(meta_json, 'r') as f:
        data = json.load(f)
    
    arrival_time_agent = data['agent']['arrival_time_ms']
    arrival_time_caller = data['caller']['arrival_time_ms']
    if arrival_time_agent < arrival_time_caller: cc+=1

    #hangup time
    hgup_time_agent = data['agent']['hangup_time_ms']
    hgup_time_caller = data['caller']['hangup_time_ms']
    if hgup_time_agent is None or hgup_time_caller is None:
        null_hg+=1
    else:
        if hgup_time_caller < hgup_time_agent: caller_hg_first+=1

    kk+=1
print(cc, kk)
print(caller_hg_first, null_hg)

0 1446
819 30


In [6]:
## 2. Created mixed audios
mixed_path = "mixed"

for idx, row in df.iterrows():
    call_id = row['sid']
    agent_wav = sc_wav.read(os.path.join(AGENT_PATH, f"{call_id}.wav"))
    caller_wav = sc_wav.read(os.path.join(CALLER_PATH, f"{call_id}.wav"))

    sampling_rate = agent_wav[0]

    ## Metadata:
    meta_json = os.path.join(NEW_METADATA_PATH, f"{call_id}.json")
    with open(meta_json, 'r') as f:
        data = json.load(f)
    start_time = data['start_time_ms']
    end_time = data['end_time_ms']
    call_duration_s = (end_time-start_time)/1000

    ## Mixed signal
    arr_size = int(sampling_rate * call_duration_s)
    mixed_arr = np.zeros(arr_size)

    ## agent arrives first always
    # caller can hangup before agent -> pad when that happens
    hgup_time_agent = data['agent']['hangup_time_ms']
    hgup_time_caller = data['caller']['hangup_time_ms']

    mixed = caller_wav[1].copy()

    if hgup_time_caller < hgup_time_agent:
        time_diff = hgup_time_agent - hgup_time_caller
        print(call_id)
    
    mixed[len(mixed)-len(agent_wav[1]):len(mixed)] += agent_wav[1]
    
    out_path = os.path.join(MIXED_PATH, f"{call_id}.wav")
    sc_wav.write(out_path, sampling_rate, mixed.astype(np.int16))


8c8a5b0505d74c5d
d902244660d44131
0a70acb6ef0c4e89
97073eb94e004260
2a01e3fdcbd44ef2
4f179504534f4600
7e864cce89594b43
78a1e380db334ee7
e31f1d8501d645fc
7df9123f42294739
eda1e39d6e724ac0
d3ea5b268ab74286
a9d903d9823b49c3
925ac203ecdc48f8
f9f57c44d80d4d75
1d412e405c724b25
b448021d143f4a37
637938e7c1194810
612cd404ade3463f
29cd0c6856ba42f6
ac5ba0e9d69d4fa0
7a81b31f172c47c2
06ff201c1fd84e1c


In [7]:
df[['sid', 'code']]

,sid,code
0,8c8a5b0505d74c5d,0
1,d902244660d44131,10
2,0a70acb6ef0c4e89,11
3,97073eb94e004260,20
4,2a01e3fdcbd44ef2,21
5,4f179504534f4600,101
6,7e864cce89594b43,110
7,78a1e380db334ee7,111
8,e31f1d8501d645fc,120
9,7df9123f42294739,121
